In [13]:
import numpy as np
import matplotlib.pyplot as plt
from Helper_Functions import Generate_Features, Quality_Evaluation
from skimage.io import imread
from skimage.transform import rescale
from sklearn.ensemble import RandomForestClassifier
from skimage.segmentation import random_walker
from multiprocessing import cpu_count
from skimage.morphology import opening, disk
import os
from sklearn.externals import joblib

In [14]:
Im_path="/home/thanos/Desktop/Project_ADAS_2017/KITI/image"
GT_path="/home/thanos/Desktop/Project_ADAS_2017/KITI/labels_3D"

def Get_Image_and_Ground_Truth(GT_path,Im_path):
    GT=[]
    Im=[]
    for file in os.listdir(GT_path):
        if file.endswith(".png"):
            GT.append(file)


    Im=os.listdir(Im_path)

    GT=sorted(GT)
    Im=sorted(Im)
    return GT,Im

In [15]:
GT,Im=Get_Image_and_Ground_Truth(GT_path,Im_path)




Features=[]
Labels=[]
#0-173
scale_factor=0.5
steps=100
for i in range(steps):
    
    Train_GT=imread(GT_path+"/"+GT[i])
    Image_Train=imread(Im_path+"/"+Im[i])
    
    # Train Image
    I_train=rescale(Image_Train,scale_factor,mode='reflect')
    nx,ny=I_train[:,:,0].shape
    # Train Labels
    Train_Labels=rescale(Train_GT,scale_factor,mode='reflect', preserve_range=True)
    mask_Train = Train_Labels == 1
    # Find Train Features 
    Train_Features=np.reshape(Generate_Features(I_train),[54,nx*ny]).T
    Features.append(Train_Features)
    Labels.append(mask_Train.reshape(-1).astype('int'))
Features=np.array(Features)
Features=Features.reshape(steps*nx*ny,54)
Labels=np.array(Labels)
Labels=Labels.reshape(steps*nx*ny)    
print("Feature and Label generation completed")

Feature and Label generation completed


In [16]:
print(np.array(Features).shape)
print(np.array(Labels).shape)
RF = RandomForestClassifier(n_estimators=128,n_jobs=cpu_count())
RF.fit(Features,Labels)

(11674800, 50)
(11674800,)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=128, n_jobs=8, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [17]:
joblib.dump(RF, 'KITI_RandomForest05.pkl') 

['KITI_RandomForest05V3.pkl']